## Computational Macroeconomics Assignment 5
#### Paula Beck - 29.01.2022

In [42]:
# some preparation

# import packages
from pathlib import Path
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols


# set path
wd = Path.cwd()

### Step 1: download data and save
downloaded on 25.01.2022

### Step 2: load data in python

In [43]:
# load consumer expectation data

# instead of changing the excel file manually, I manipulate the specific data directly in python by loading the explicite sheet and skipping the first three rows. Also, I only load the first three columns as those are the one we need.
exp = pd.read_excel(wd/'data'/'FRBNY-SCE-Data.xlsx', sheet_name = 'Inflation expectations', skiprows=3, usecols=[0,1,2])

exp.rename(columns={exp.columns[0]:'date'}, inplace=True)

exp.head()

,date,Median one-year ahead expected inflation rate,Median three-year ahead expected inflation rate
0,201306,3.090884,3.416846
1,201307,3.162522,3.310545
2,201308,3.395071,3.799491
3,201309,3.367290,3.546918
4,201310,3.174733,3.196597


In [44]:
# load inflation data

inf = pd.read_csv(wd/'data'/'CPALTT01USM659N.csv', sep = ',', decimal ='.', names=['date', 'inflation'], header=0)

inf.head()

,date,inflation
0,1960-01-01,1.034483
1,1960-02-01,1.730104
2,1960-03-01,1.730104
3,1960-04-01,1.724138
4,1960-05-01,1.724138


### Step 3: convert date columns to datetime-type

In [45]:
# first look what data types they have
print('---data types of inflation data------------')
print(inf.dtypes)
print('---data types of expectation data----------')
print(exp.dtypes)

---data types of inflation data------------
date          object
inflation    float64
dtype: object
---data types of expectation data----------
date                                                 int64
Median one-year ahead expected inflation rate      float64
Median three-year ahead expected inflation rate    float64
dtype: object


In [46]:
# convert date column of inflation dataframe to datetime
inf.date  = pd.to_datetime(inf.date)
# set frequency to monthly
inf.date = pd.PeriodIndex(inf.date, freq='m')

# convert the date column of the expectation dataframe to a string and add '01', then convert to datetime
exp.date = exp.date.astype('str') + '01'
exp.date = pd.to_datetime(exp.date, format = '%Y%m%d')
# set frequency to monthly
exp.date = pd.PeriodIndex(exp.date, freq='m')


In [47]:
# check data types again
print('---data types of inflation data------------')
print(inf.dtypes)
print('---data types of expectation data----------')
print(exp.dtypes)

---data types of inflation data------------
date         period[M]
inflation      float64
dtype: object
---data types of expectation data----------
date                                               period[M]
Median one-year ahead expected inflation rate        float64
Median three-year ahead expected inflation rate      float64
dtype: object


In [48]:
# set date as index
inf = inf.set_index('date')
exp = exp.set_index('date')

### Step 4: rename expectation columns

In [49]:
exp.rename(columns={exp.columns[0]:'med1y',exp.columns[1]:'med3y'}, inplace=True)

### Step 5: merge dataframes

In [50]:
# merge expectation data to inflation dataframe based on the index
# inner merge such that only those rows remain that exist for both data sources
df = inf.merge(exp, on='date', how='inner')
df.head()

,inflation,med1y,med3y
date,,,
2013-06,1.754417,3.090884,3.416846
2013-07,1.960682,3.162522,3.310545
2013-08,1.518368,3.395071,3.799491
2013-09,1.184925,3.367290,3.546918
2013-10,0.963613,3.174733,3.196597


### Step 6: calculate forecast errors and forecast revision

In [51]:
# set all to point t

df['ferror'] = df.inflation.shift(-12) - df.med1y 
df['frevision'] = df.med1y - df.med3y.shift(24) 

### Step 7: drop all columns except from forecast errors and forecast revision

In [52]:
# keep only those two columns
df = df[['ferror', 'frevision']]
df.head()

,ferror,frevision
date,,
2013-06,-1.018543,NaN
2013-07,-1.170193,NaN
2013-08,-1.695460,NaN
2013-09,-1.709371,NaN
2013-10,-1.510392,NaN


### Step 8: remove NAN values

In [53]:
df = df.dropna()
print(df.shape)

(67, 2)


### Step 9: estimate equation (1)

- equation: ferror = constant + frevision + error
- set cov_type = 'HAC'
- set cov_kwds = {'maxlags':3}

In [54]:
model = ols(formula='ferror ~ frevision', data=df)
fit = model.fit(cov_type = 'HAC', cov_kwds = {'maxlags':3})
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ferror   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.1166
Date:                Wed, 26 Jan 2022   Prob (F-statistic):              0.734
Time:                        17:05:25   Log-Likelihood:                -117.13
No. Observations:                  67   AIC:                             238.3
Df Residuals:                      65   BIC:                             242.7
Df Model:                           1                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3265      0.403     -0.810      0.418      -1.117       0.464
frevision      0.2154      0.631      0.341      0.733      -1.021       1.452
==============================================================================
Omnibus:                       24.819   Durbin-Watson:                   0.088
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.693
Skew:                           1.516   Prob(JB):                     1.78e-08
Kurtosis:                       4.895   Cond. No.                         3.16
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 3 lags and without small sample correction
"""

### Step 10: Interpretation

$\beta$ can be interpreted as the relation between the forecast error and the forecast revision. Under rational expectations, the coefficient should be not significantly different from zero, i.e. the updating process of the forecast should not affect the forecast error. We find that this in indeed the case. Coibion and Gorodnychenko (2015), henceforth CG, however, find a significant correlation between the two, such that the forecast error can be (partly) explained by the forecast revision. A possible explanation for the difference is that we have a different dataset and a small sample size (67 observations). Also, CG use a survey of professional forecasters while our dataset consists of households inflation expectations.